In [1]:
import gym
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

%matplotlib inline
sns.set_style("whitegrid")
sns.set_palette("colorblind")

# Часть первая
## Пункт 1
>Рассмотрим очень простую стратегию: говорить stand, если у нас на руках комбинация в 19, 20 или 21 очко, во всех остальных случаях говорить hit. Используйте методы Монте-Карло, чтобы оценить выигрыш от этой стратегии.  

Это эпизодическая задача, причём средняя длина эпизода небольшая (единицы ходов), поэтому положим $\gamma = 1.$ По этой причине return ($G_t$) на каждом ходу будет просто равен reward на последнем ходу.  
Самый простой способ оценить средний выигрыш стратегии - сгенерировать большое число эпизодов и усреднить выигрыш. Так и сделаем.

In [2]:
env = gym.make('Blackjack-v0', natural=True)

In [3]:
class BaselineStrategy:
    def __init__(self):
        pass
    
    def get_action(self, state):
        player_sum, dealer_card, usable_ace = state
        if player_sum < 19:
            return 1  # hit
        return 0  # stand

In [4]:
baseline = BaselineStrategy()

In [5]:
def run_episode(env, strategy):
    state = env.reset()
    while True:
        action = strategy.get_action(state)
        state, reward, done, _ =  env.step(action)
        if done:
            return reward

def eval_strategy(env, strategy, num_iterations):
    return np.mean([run_episode(env, strategy) for _ in trange(num_iterations)])

In [6]:
eval_strategy(env, baseline, 5000000)

100%|█████████████████████████████████████████████████████████████████████| 5000000/5000000 [04:33<00:00, 18261.76it/s]


-0.1806987

Но ничто нам не мешает немного перемудрить и построить Monte-Carlo estimation функции $V_{\pi}(s)$, а затем взять её матожидание по распределению начальных состояний (которое тоже можно оценить методом Монте-Карло).  
Для данной стратегии имеет значение лишь общая сумма на руках, но с суммой 21 нужно поступить хитрее: в начальном состоянии 21 можно получить только в виде пары (Ace, 10), что есть Natural blackjack и даёт reward=1.5 (если дилер тоже не наберёт 21, тогда reward=0), так что состоянием будем считать только сумму очков на руках, но 21 разобьём на 2 состояния (Natural blackjack или нет).  
Оценим вероятности начальных состояний:  

In [7]:
from collections import Counter
starts_counter = Counter(env.reset()[0] for _ in trange(5000000))

100%|█████████████████████████████████████████████████████████████████████| 5000000/5000000 [02:52<00:00, 29035.86it/s]


In [8]:
starts_frequencies = pd.Series(starts_counter).sort_index()
starts_frequencies /= starts_frequencies.sum()
starts_frequencies

4     0.005900
5     0.011858
6     0.017857
7     0.023689
8     0.029617
9     0.035516
10    0.041429
11    0.047430
12    0.094695
13    0.094860
14    0.088761
15    0.082924
16    0.076803
17    0.070999
18    0.065013
19    0.058906
20    0.106309
21    0.047434
dtype: float64

In [9]:
class RunningMeans:
    def __init__(self, shape):
        self.means = np.zeros(shape)
        self.sizes = np.zeros(shape)
    
    def update(self, index, value):
        m = self.means[index]
        n = self.sizes[index]
        self.means[index] = (m * n + value) / (n + 1)
        self.sizes[index] += 1

def get_states_and_reward(env, strategy):
    state = env.reset()
    states = []
    while True:
        states.append(state)
        action = strategy.get_action(state)
        state, reward, done, _ =  env.step(action)
        if done:
            return states, reward

def estimate_V_for_baseline(env, strategy, num_iterations):
    V = RunningMeans(env.observation_space[0].n)
    for _ in trange(num_iterations):
        #  Поскольку gamma==1, можно брать reward с последнего шага в качестве return на всех шагах,
        #  а также обходить эпизод от начала к концу, а не наоборот
        states, reward = get_states_and_reward(env, strategy)
        for player_sum, dealer_card, usable_ace in states:
            if player_sum == 21:
                if len(states) == 1:  # Natural blackjack; будем его записывать в ячейку 21
                    V.update(21, reward)
                else:  # 21 получено другим способом; сохраним отдельно в ячейку 22 - такой суммы всё равно не может быть
                    V.update(22, reward)
            else:
                V.update(player_sum, reward)
    return V

In [10]:
#  Получить меньше 4 и больше 21 очков невозможно
#  В ячейке 21 хранится оценка для состояния Natural blackjack
#  В ячейке 22 хранится оценка для суммы 21, полученной иным способом
baseline_V = estimate_V_for_baseline(env, baseline, 5000000).means[4:23]
baseline_V

100%|█████████████████████████████████████████████████████████████████████| 5000000/5000000 [05:28<00:00, 15204.96it/s]


array([-0.34953906, -0.36656564, -0.38691354, -0.41579624, -0.41458222,
       -0.16153505, -0.02335208,  0.04528941, -0.41877361, -0.44277624,
       -0.47682696, -0.51144324, -0.54092291, -0.5692912 , -0.59529848,
        0.26731447,  0.58007819,  1.31888521,  0.87947545])

Для Natural blackjack оценка не равна 1.5, потому что дилер тоже может набрать 21 и будет ничья

In [11]:
(starts_frequencies.values * baseline_V[:-1]).sum()

-0.1870863467778301